In [2]:
import cv2
from ultralytics import YOLO
import os
import numpy as np

# YOLO 모델 로드
model = YOLO('best32_40.pt')  # YOLOv8 모델 경로로 변경

# 얼굴 인식 결과 시각화 함수
def draw_boxes(results, img):
    for result in results:
        boxes = result.boxes.xyxy  # x1, y1, x2, y2 format
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return img

# 얼굴 영역에만 가우시안 블러 처리 함수
def blur_faces(results, img):
    mask = np.zeros_like(img)
    for result in results:
        boxes = result.boxes.xyxy  # x1, y1, x2, y2 format
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = img[y1:y2, x1:x2]
            blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
            mask[y1:y2, x1:x2] = blurred_face
    # 원본 이미지와 블러 처리된 영역 합성
    img = np.where(mask == 0, img, mask)
    return img

# 얼굴 블러 처리 함수
def blur_faces(results, img, blur_size=(35, 25)):
    for result in results:
        boxes = result.boxes.xyxy  # x1, y1, x2, y2 format
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = img[y1:y2, x1:x2]
            
            # 얼굴 영역 마스크 생성
            mask = np.zeros_like(face)
            center = (mask.shape[1] // 2, mask.shape[0] // 2)
            axes = (mask.shape[1] // 2, mask.shape[0] // 2)
            cv2.ellipse(mask, center, axes, 0, 0, 360, (255, 255, 255), -1)
            
            # 얼굴 영역 블러 처리
            blurred_face = cv2.GaussianBlur(face, blur_size, 0)
            face = np.where(mask == 255, blurred_face, face)
            img[y1:y2, x1:x2] = face
    return img


# 동영상 처리 함수
def process_video(video_path):
    print(f"Processing video: {video_path}")
    
    # 동영상 읽기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    # 동영상 저장 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 동영상 저장 경로 설정
    detected_output_path = video_path.replace('.mp4', '_detected.mp4')
    blurred_output_path = video_path.replace('.mp4', '_blurred.mp4')
    
    # 동영상 저장 객체 생성
    out_detected = cv2.VideoWriter(detected_output_path, fourcc, fps, (width, height))
    out_blurred = cv2.VideoWriter(blurred_output_path, fourcc, fps, (width, height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}")

        # 추론
        results = model(frame, conf=0.25, imgsz=1280, max_det=1000)

        # 얼굴 인식 결과 시각화
        detected_frame = draw_boxes(results, frame.copy())
        out_detected.write(detected_frame)

        # 얼굴 영역에만 가우시안 블러 처리
        blurred_frame = blur_faces(results, frame.copy())
        out_blurred.write(blurred_frame)

    # 자원 해제
    cap.release()
    out_detected.release()
    out_blurred.release()

    print(f"Processed video saved as {detected_output_path} and {blurred_output_path}")

    # 결과 동영상 시스템 기본 동영상 플레이어로 열기
    if os.name == 'posix':  # MacOS
        os.system(f'open {detected_output_path}')
        os.system(f'open {blurred_output_path}')
    elif os.name == 'nt':  # Windows
        os.system(f'start {detected_output_path}')
        os.system(f'start {blurred_output_path}')
    else:  # Linux
        os.system(f'xdg-open {detected_output_path}')
        os.system(f'xdg-open {blurred_output_path}')

    print(f"{video_path} 동영상 처리가 완료되었습니다.")

# 동영상 파일 처리
video_files = [f for f in os.listdir() if f.endswith('.mp4')]
print(f"Found {len(video_files)} video files.")
for video_file in video_files:
    process_video(video_file)


Found 1 video files.
Processing video: test3.mp4

0: 864x1280 14 faces, 370.3ms
Speed: 9.0ms preprocess, 370.3ms inference, 0.8ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 13 faces, 364.7ms
Speed: 7.7ms preprocess, 364.7ms inference, 0.6ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 12 faces, 365.1ms
Speed: 8.5ms preprocess, 365.1ms inference, 0.6ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 13 faces, 363.9ms
Speed: 8.1ms preprocess, 363.9ms inference, 0.6ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 13 faces, 366.4ms
Speed: 8.0ms preprocess, 366.4ms inference, 0.7ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 13 faces, 368.7ms
Speed: 8.3ms preprocess, 368.7ms inference, 0.7ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 14 faces, 387.6ms
Speed: 8.4ms preprocess, 387.6ms inference, 0.6ms postprocess per image at shape (1, 3, 864, 1280)

0: 864x1280 14 faces, 364.6ms
Speed: 8.2